In [1]:
df = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferschema","true")\
    .load("s3://project.group23.fan5.flight/Scaled_Data_M/")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1648292310456_0004,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

60689539

In [3]:
from pyspark.sql.functions import col, column

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#test data on 2018 

trainDF = df.where(col("YEAR")!=2018)
testDF = df.where(col("YEAR")==2018)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
trainDF.printSchema() , testDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- OP_CARRIER: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- WEEKDAY: integer (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- FLIGHT_STATUS: integer (nullable = true)
 |-- OP_CARRIER_I: double (nullable = true)
 |-- ORIGIN_I: double (nullable = true)
 |-- DEST_I: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- indexed_features: vector (nullable = true)
 |-- scaledfeatures: vector (nullable = true)

root
 |-- OP_CARRIER: string (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (n

In [7]:
# print the count of observations in each set
print("Observations in training set = ", trainDF.count())
print("Observations in testing set = ", testDF.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Observations in training set =  53613134
Observations in testing set =  7076405

In [8]:
trainDF, testDF1 =  trainDF.randomSplit([0.5,0.5], seed = 143)
trainDF1, testDF =  testDF.randomSplit([0.5,0.5], seed = 143)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# print the count of observations in each set
print("Observations in training set = ", trainDF.count())
print("Observations in testing set = ", testDF.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Observations in training set =  26811048
Observations in testing set =  3539398

In [10]:
# import the RandomForestClassifier function from the pyspark.ml.classification package
from pyspark.ml.classification import RandomForestClassifier

# Build the RandomForestClassifier object 'dt' by setting the required parameters
# We will pass the VectorIndexed columns as featureCol for Random Forest. Since they can handle categorical indexes
rf = RandomForestClassifier(featuresCol="scaledfeatures", labelCol="FLIGHT_STATUS" , numTrees=50, maxDepth= 5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# fit the RandomForestClassifier object on the training data
rfmodel = rf.fit(trainDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#This RandomForestClassifierModel can be used as a transformer to perform prediction on the testing data
rfpredictonDF = rfmodel.transform(testDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
rfpredictonDF.select("FLIGHT_STATUS","rawPrediction", "probability", "prediction").show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+---------------------------------------+----------------------------------------+----------+
|FLIGHT_STATUS|rawPrediction                          |probability                             |prediction|
+-------------+---------------------------------------+----------------------------------------+----------+
|0            |[32.98429880816329,17.01570119183672]  |[0.6596859761632656,0.3403140238367343] |0.0       |
|1            |[29.036990145513208,20.963009854486792]|[0.5807398029102642,0.41926019708973583]|0.0       |
|1            |[29.363585599329337,20.636414400670656]|[0.5872717119865868,0.41272828801341316]|0.0       |
|1            |[29.363585599329337,20.636414400670656]|[0.5872717119865868,0.41272828801341316]|0.0       |
|1            |[29.363585599329337,20.636414400670656]|[0.5872717119865868,0.41272828801341316]|0.0       |
|0            |[29.036990145513208,20.963009854486792]|[0.5807398029102642,0.41926019708973583]|0.0       |
|0            |[29.363585599

In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
invalid syntax (<stdin>, line 1)
  File "<stdin>", line 1
    from pyspark.ml.evaluation import multievaluator.evaluate
                                                    ^
SyntaxError: invalid syntax



In [16]:
# 1. Accuracy
print("Accuracy: ", multievaluator.evaluate(rfpredictonDF, {evaluator.metricName: "accuracy"})) 
# 2. Area under the ROC curve
print('Area under the ROC curve = ', evaluator.evaluate(rfpredictonDF))
# 3. Precision (Positive Predictive Value)
print("Precision = ", multievaluator.evaluate(rfpredictonDF, {evaluator.metricName: "weightedPrecision"}))
# 4. Recall (True Positive Rate)
print("Recall = ", multievaluator.evaluate(rfpredictonDF, {evaluator.metricName: "weightedRecall"}))
# 5. F1 Score (F-measure)
print("F1 Score = ", multievaluator.evaluate(rfpredictonDF, {evaluator.metricName: "f1"}))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'multievaluator' is not defined
Traceback (most recent call last):
NameError: name 'multievaluator' is not defined



In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='FLIGHT_STATUS', 
    predictionCol='prediction', 
    metricName='accuracy')
accuracy = evaluator.evaluate(rfpredictonDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# 1. Accuracy
print("Accuracy: ", accuracy)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy:  0.6487464817463309

In [23]:
# 5. F1 Score (F-measure)
print("F1 Score = ", evaluator.evaluate(rfpredictonDF, {evaluator.metricName: "f1"}))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

F1 Score =  0.5346869982197521

In [26]:
# import the RandomForestClassifier function from the pyspark.ml.classification package
from pyspark.ml.classification import RandomForestClassifier

# Build the RandomForestClassifier object 'dt' by setting the required parameters
# We will pass the VectorIndexed columns as featureCol for Random Forest. Since they can handle categorical indexes
rf1 = RandomForestClassifier(featuresCol="scaledfeatures", labelCol="FLIGHT_STATUS" , numTrees=75, maxDepth= 5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# fit the RandomForestClassifier object on the training data
rfmodel1 = rf1.fit(trainDF)

An error was encountered:
Session 3 did not reach idle status in time. Current status is busy.


In [ ]:
#This RandomForestClassifierModel can be used as a transformer to perform prediction on the testing data
rfpredictonDF_1 = rfmodel1.transform(testDF)